In [ ]:
import cv2
import numpy as np
import pandas as pd
from keras.layers import Cropping2D
from scipy import ndimage
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder

factor = 1.0 / 4.0

def resize(image):
    return cv2.resize(image, dsize=None, fx=factor, fy=factor)


def preprocess(image):
    return resize(image)


def create_Cropping2D():
    return Cropping2D(cropping=((int(70 * factor), int(25 * factor)), (0, 0)))


def get_input_shape():
    width = 800
    height = 600
    return (int(height * factor), int(width * factor), 3)


def get_images_and_measurements():
    def read_and_preprocess(image_file):
        return preprocess(ndimage.imread(image_file))
    
    items = ['images/img_0001_RED.jpg', 'images/img_0002_RED.jpg', 'images/img_0168_GREEN.jpg', 'images/img_0205_NO_TRAFFIC_LIGHT.jpg']
    images = list(map(read_and_preprocess, items))
    # encode class values as integers
    encoder = LabelEncoder()
    Y = ['RED', 'RED', 'GREEN', 'NO_TRAFFIC_LIGHT']
    encoder.fit(Y)
    encoded_Y = encoder.transform(Y)
    return images, np_utils.to_categorical(encoded_Y, 3)

def flip_image(image):
    return np.fliplr(image)


def flip_measurement(measurement):
    return -measurement


def flip_images(images):
    return map(flip_image, images)


def flip_measurements(measurements):
    return map(flip_measurement, measurements)


def get_steering_left(steering_center):
    return steering_center + 0.2


def get_steering_right(steering_center):
    return steering_center - 0.2

In [ ]:
import numpy as np
from keras.callbacks import ModelCheckpoint
from keras.layers import Flatten, Dense, Lambda, MaxPooling2D
from keras.layers.convolutional import Convolution2D
from keras.models import Sequential

def get_X_train_y_train():
    images, measurements = get_images_and_measurements()
    X_train = np.array(images)
    y_train = np.array(measurements)
    return X_train, y_train


def create_model_simple():
    model = Sequential()
    model.add(Flatten(input_shape=get_input_shape()))
    model.add(Dense(3, activation="softmax"))
    return model


def create_model_LeNet():
    model = Sequential()
    model.add(Lambda(lambda image: image / 255.0 - 0.5, input_shape=get_input_shape()))
    model.add(create_Cropping2D())
    model.add(Convolution2D(6, 5, 5, activation='relu'))
    model.add(MaxPooling2D())
    model.add(Convolution2D(6, 5, 5, activation='relu'))
    model.add(MaxPooling2D())
    model.add(Flatten())
    model.add(Dense(120))
    model.add(Dense(84))
    model.add(Dense(3, activation="softmax"))
    return model


def create_model_Nvidia():
    model = Sequential()
    model.add(Lambda(lambda image: image / 255.0 - 0.5, input_shape=get_input_shape()))
    model.add(create_Cropping2D())
    model.add(Convolution2D(24, 5, 5, subsample=(2, 2), activation='relu'))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(Convolution2D(64, 3, 3, activation='relu'))
    model.add(Flatten())
    model.add(Dense(100))
    model.add(Dense(50))
    model.add(Dense(10))
    model.add(Dense(3, activation="softmax"))
    return model


def train(model, X, y, save_model_2_file):
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    history_object = model.fit(X,
                               y,
                               validation_split=0.2,
                               shuffle=True,
                               callbacks=[ModelCheckpoint(filepath=save_model_2_file, verbose=1, save_best_only=True)],
                               epochs=5,
                               verbose=1)
    return history_object

In [ ]:
import matplotlib.pyplot as plt
# Visualizations will be shown in the notebook.
%matplotlib inline

In [ ]:
def plot_model(model, to_file):
    from keras.utils import plot_model
    from IPython.display import Image
    plot_model(model, show_shapes=True, show_layer_names=False, rankdir='TB', to_file=to_file)
    return Image(filename=to_file)

In [ ]:
model = create_model_Nvidia()
plot_model(model, "examples/model_nvidia.jpg")

In [ ]:
import numpy as np
from scipy import ndimage
from scipy.misc import imsave

image = ndimage.imread('images/img_0001_RED.jpg')
plt.imshow(image)
plt.show()

preprocessed_image = preprocess(image)
plt.imshow(preprocessed_image)
imsave('examples/center_2016_12_01_13_31_15_308_quartered.jpg', preprocessed_image)
plt.show()

In [ ]:
def plot_loss(history_object, savefile):
    plt.plot(history_object.history['loss'])
    plt.plot(history_object.history['val_loss'])
    plt.title('model mean squared error loss')
    plt.ylabel('mean squared error loss')
    plt.xlabel('epoch')
    plt.legend(['training set', 'validation set'], loc='upper right')
    plt.savefig(savefile)
    plt.show()

In [ ]:
X_train, y_train = get_X_train_y_train()

In [ ]:
y_train

In [ ]:
def train_and_plot(model, X, y, save_model_2_file, save_fig_2_file):
    history_object = train(model, X, y, save_model_2_file)
    plot_loss(history_object, save_fig_2_file)
    print("minimal validation loss: {:.5f}".format(min(history_object.history['val_loss'])))

In [ ]:
train_and_plot(create_model_simple(), X_train, y_train, save_model_2_file='model_simple.h5', save_fig_2_file='examples/loss_model_simple.png')

In [ ]:
train_and_plot(create_model_LeNet(), X_train, y_train, 'model_lenet.h5', save_fig_2_file='examples/loss_model_lenet.png')

In [ ]:
train_and_plot(create_model_Nvidia(), X_train, y_train, 'model_nvidia.h5', save_fig_2_file='examples/loss_model_nvidia.png')